In [1]:
import numpy as np
import pandas as pd
import tqdm 
import cProfile
import lightgbm as lgb
from sklearn.model_selection import GroupKFold, train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import optuna
import logging
import time
from scipy.stats import pearsonr

data = pd.read_feather("data/processed/massBV.feather")

In [2]:
def prep_data_before_train(data: pd.DataFrame, phenotype: str) -> tuple:
    """
    prepare data for training, returns target vector and covariate-matrix and ringnrs for grouping
    :param data: all data from dataloader script
    :param phenotype: the phenotype to be predicted
    :return: X, Y, ringnrs, X contain covariates and Y contains the phenotype
    """
    ringnrs = data.ringnr
    mean_pheno = data.mean_pheno
    X = data.drop(
        columns=[
            "ID",
            "mass",
            "tarsus",
            "ringnr",
            "mean_pheno",
            "IID",
            "FID",
            "MAT",
            "PAT",
            "SEX",
            "PHENOTYPE",
        ],
        errors="ignore",
    )

    try:
        Y = data.loc[:, phenotype]
    except KeyError:
        try:
            Y = data.ID
        except AttributeError:
            Y = data.mass
    del data
    try:
        X.hatchyear = X.hatchyear.astype("int")
        X.island_current = X.island_current.astype("int")
    except AttributeError:
        pass
    Y = (Y - np.mean(Y))/np.std(Y)
    X = X.fillna(0)
    X = X.T
    X = X.astype("int")
    X = X.T 
    return X, Y, ringnrs, mean_pheno

In [3]:
X, y, ringnrs, mean_pheno = prep_data_before_train(data, "mass")
del data
X.drop(columns = ["hatchisland"], inplace = True)
X["ringnr"] = ringnrs   

target = pd.DataFrame(y)
target["mean_pheno"] = mean_pheno
target["ringnr"] = ringnrs

In [4]:
def random_split(pheno:str = "mass", num_folds:int = 5, seed: int = 42) -> None:
    data = pd.read_feather(f"data/processed/{pheno}BV.feather")  
    x = data[["ringnr","hatchisland"]]
    del data
    np.random.seed(seed)
    #Split the data into num_folds folds with random sampling, each fold should have the same number of individuals
    #The folds should be saved in a new column called "fold" in the data frame

    x = x.sample(frac=1, random_state=seed).reset_index(drop=True)
    # Number of folds
    n_folds = num_folds

    # Create an array with fold numbers
    folds = np.array([i % n_folds for i in range(len(x))])

    # Shuffle the fold assignment array
    #np.random.shuffle(folds)

    # Assign the fold numbers to a new column in the dataframe
    x['fold'] = folds

    return x[['ringnr','fold']]

splits = random_split("mass", 10, 42)

In [5]:
X = pd.merge(X,splits, on = "ringnr", how = "inner") 
X = pd.merge(X,target, on = "ringnr", how = "inner")  

In [6]:
def subset(X:pd.DataFrame, seed:int = 42, snp_prop:float = 0.2) -> pd.DataFrame:
    """
    Function to subset the data into a smaller dataset for testing purposes, 
    sampling only a subset of the features
    """
    np.random.seed(seed)
    sample_columns = np.random.choice(X.columns[:-4], int(len(X.columns[:-4])*snp_prop), replace = False)
    sample_columns = np.append(sample_columns, ["ringnr","fold", "ID", "mean_pheno"])

    return X[sample_columns]

In [7]:
X = subset(X, snp_prop = 0.1)

In [8]:
X

,SNPa352024_T,SNPa51204_C,SNPa371349_T,SNPa86522_T,SNPa395916_T,SNPa302259_A,SNPa524601_A,SNPa483891_T,SNPa207156_T,SNPa77054_G,...,SNPa437479_G,SNPa179086_T,SNPa392716_T,SNPa415615_G,SNPa280074_G,SNPa220371_G,ringnr,fold,ID,mean_pheno
0,0,0,1,2,1,0,0,1,0,0,...,1,1,1,1,0,0,8118424,8,0.519577,32.600000
1,0,0,1,2,0,0,0,1,0,1,...,1,0,1,1,1,0,8118425,4,-1.044296,30.300000
2,0,0,0,1,0,0,0,0,0,1,...,2,1,1,0,0,1,8118426,7,0.918158,33.316667
3,0,0,0,0,0,0,0,1,0,1,...,1,1,1,1,0,0,8118429,1,-0.093239,31.300000
4,0,0,0,2,1,0,0,1,0,0,...,1,2,1,1,0,0,8118430,9,-1.010615,29.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1913,0,0,0,0,0,0,0,0,0,1,...,1,1,0,0,1,0,8N87368,5,0.523029,32.900000
1914,0,0,0,1,0,0,1,0,1,1,...,0,2,0,0,1,0,8N87369,5,-0.241355,30.900000
1915,0,0,0,0,1,1,0,0,1,2,...,1,1,0,0,0,1,8N87371,7,0.515544,32.500000
1916,0,0,0,1,0,0,0,0,1,1,...,0,1,0,0,0,1,8N87372,3,0.494145,32.800000


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
# Replace with your model

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Assuming your dataframe 'df' has a column 'island' with values 1 to 8 representing the islands
# and a 'fold' column representing the 8 folds, and a 'target' column for the target variable

# Function to perform the nested cross-validation
def nested_cv(df):
    
    #outer_results = []

    
    # Define the Optuna objective function
    def objective(trial):
        logger.info(f"Starting trial {trial.number}")

        # Define hyperparameter search space
        n_estimators = trial.suggest_int('n_estimators', 300, 1000)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
        max_depth = trial.suggest_int('max_depth', 3, 13)
        num_leaves = trial.suggest_int('num_leaves', 20, 3000, step = 20)
        reg_lambda = trial.suggest_int('reg_lambda', 0, 100, step = 5)
        reg_alpha = trial.suggest_int('reg_alpha', 0 , 100, step = 5)
        bagging_fraction = trial.suggest_float('bagging_fraction', 0.2, 0.9, step = 0.1)
        bagging_freq = trial.suggest_int('bagging_freq', 1, 10, step = 1)
        feature_fraction = trial.suggest_float('feature_fraction', 0.2, 0.9, step = 0.1)

        #parameters
        params = {
            'objective': 'regression',
            'metric': 'mae',    
            #'n_estimators': n_estimators,
            'learning_rate': learning_rate,
            'max_depth': max_depth,
            'num_leaves': num_leaves,
            'reg_lambda': reg_lambda,
            'reg_alpha': reg_alpha,
            'bagging_fraction': bagging_fraction,
            'bagging_freq': bagging_freq,
            'feature_fraction': feature_fraction,
            'verbose': -1
        }

        outer_results = []  
        folds = np.sort(df['fold'].unique())    
        for outer_fold in folds:
            #print(f"Outer CV: Test island {outer_fold}")
        
            # Split data into outer train and test sets
            outer_train_df = df[df['fold'] != outer_fold]
            outer_test_df = df[df['fold'] == outer_fold]

            # Extract features and target for outer training and test sets
            X_train_outer, y_train_outer = outer_train_df.drop(columns=['ringnr', 'fold','ID','mean_pheno']), outer_train_df['ID']
            X_test_outer, y_test_outer = outer_test_df.drop(columns=['ringnr', 'fold','ID','mean_pheno']), outer_test_df['mean_pheno']

            #split X_train_outer and y_train_outer into train and validation set
            X_train_outer, X_val_outer, y_train_outer, y_val_outer = train_test_split(X_train_outer, y_train_outer, test_size = 0.1, random_state = 42)

            # Train on the outer train set
            dtrain = lgb.Dataset(X_train_outer, label=y_train_outer)
            dval = lgb.Dataset(X_val_outer, label=y_val_outer, reference=dtrain)

            gbm = lgb.train(params, dtrain, num_boost_round=n_estimators, valid_sets=[dtrain,dval], valid_names=['train', 'valid'], callbacks=[lgb.early_stopping(stopping_rounds=100)])

            # Validate model
            y_pred_outer = gbm.predict(X_test_outer, num_iteration = gbm.best_iteration)
            corr, _ = pearsonr(y_test_outer, y_pred_outer)
            #print(y_pred_outer)
            #print()
            #print(y_test_outer)

            #print(f"Test correlation for island {outer_fold}: {outer_corr}")
            outer_results.append(corr)
        
        # Final results
        print(f"Results: {outer_results}")
        print(f"Average correlation across all outer folds: {np.mean(outer_results)}")
        return np.mean(outer_results)
    
    # Run Optuna for hyperparameter optimization
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=15, n_jobs=4)

    return study


s = nested_cv(X)

            # Perform 7-fold CV on the 7 remaining islands
        #    inner_corr = []
        #    start_time = time.time()
        #    for inner_fold in folds:
        #        if inner_fold == outer_fold:
        #            continue  # Skip the current test island
        #        
        #        # Split inner train/validation data
        #        train_df = outer_train_df[outer_train_df['fold'] != inner_fold]
        #        val_df = outer_train_df[outer_train_df['fold'] == inner_fold]
        #        
        #        # Extract features and target
        #        X_train, y_train = train_df.drop(columns=['ringnr', 'fold','ID','mean_pheno']), train_df['ID']
        #        X_val, y_val = val_df.drop(columns=['ringnr', 'fold','ID','mean_pheno']), val_df['mean_pheno'] #Use this for computing Pearson correlation, this will be the performance metric for the inner loop#
        #
        #        # Train model
        #        dtrain = lgb.Dataset(X_train, label=y_train)
        #        dval = lgb.Dataset(X_val, label=y_val, reference=dtrain)

        #        gbm = lgb.train(params, dtrain, num_boost_round=n_estimators, valid_sets=[dval], callbacks=[lgb.early_stopping(stopping_rounds=20)])
                # Validate model
        #        y_pred = gbm.predict(X_val)
        #        corr = np.corrcoef(y_val, y_pred)[0, 1]
        #        inner_corr.append(corr)
                
        #        end_time = time.time()
        #    logger.info(f"Finished trial {trial.number} in {end_time - start_time:.2f} seconds")
            # Return the average MAE for this set of hyperparameters
        #    return np.mean(inner_corr)
        
        # Run Optuna for hyperparameter optimization
        #study = optuna.create_study(direction='maximize')
        #study.optimize(objective, n_trials=10, n_jobs=4)
        
        # Train the final model on the outer train set with the best hyperparameters
        #best_params = study.best_params

        #n_estimators = best_params['n_estimators']
        #best_params.pop('n_estimators')
        #best_params['objective'] = 'regression'
        #best_params['metric'] = 'mae'
        #best_params['verbose'] = -1
        
        # Extract features and target for outer training and test sets
        #X_train_outer, y_train_outer = outer_train_df.drop(columns=['ringnr', 'fold','ID','mean_pheno']), outer_train_df['ID']
        #X_test_outer, y_test_outer = outer_test_df.drop(columns=['ringnr', 'fold','ID','mean_pheno']), outer_test_df['mean_pheno']
        
        # Train on the outer train set
        #dtrain = lgb.Dataset(X_train_outer, label=y_train_outer)
        #dval = lgb.Dataset(X_test_outer, label=y_test_outer, reference=dtrain)  
        #gbm = lgb.train(best_params, dtrain, num_boost_round=n_estimators, valid_sets=[dval], callbacks=[lgb.early_stopping(stopping_rounds=20)])
        
        # Test on the outer test set
        #y_pred_outer = gbm.predict(X_test_outer)
        #outer_corr = np.corrcoef(y_test_outer, y_pred_outer)[0, 1]
        
        #print(f"Test correlation for island {outer_fold}: {outer_corr}")
        #outer_results.append(outer_corr)
    
    # Final results
    #print(f"Average correlation across all outer folds: {np.mean(outer_results)}")
    #return outer_results

# Example dataframe with 'island', 'fold', 'target', and other features



[I 2024-09-26 11:51:18,844] A new study created in memory with name: no-name-2e9c6589-7056-44f7-96ff-1052c4d23999
INFO:__main__:Starting trial 0
INFO:__main__:Starting trial 1
INFO:__main__:Starting trial 2
INFO:__main__:Starting trial 3


Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[16]	train's l1: 0.770693	valid's l1: 0.798822
Early stopping, best iteration is:
[1]	train's l1: 0.771664	valid's l1: 0.799257


C:\Users\simen\AppData\Local\Temp\ipykernel_41008\1244612948.py:72: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = pearsonr(y_test_outer, y_pred_outer)


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[173]	train's l1: 0.585095	valid's l1: 0.757215
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	train's l1: 0.766692	valid's l1: 0.74104


C:\Users\simen\AppData\Local\Temp\ipykernel_41008\1244612948.py:72: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = pearsonr(y_test_outer, y_pred_outer)


...

[I 2024-09-26 12:16:31,996] Trial 13 finished with value: 0.19093623705651203 and parameters: {'n_estimators': 970, 'learning_rate': 0.21648811107927657, 'max_depth': 8, 'num_leaves': 2340, 'reg_lambda': 10, 'reg_alpha': 40, 'bagging_fraction': 0.5, 'bagging_freq': 9, 'feature_fraction': 0.6000000000000001}. Best is trial 13 with value: 0.19093623705651203.


Results: [0.24395886727101251, 0.20053764576967664, 0.13816775965693556, 0.12056817361086034, 0.23817623221255418, 0.14713461827405977, 0.15620906118736821, 0.11793592983394259, 0.23105331025778333, 0.31562077249092707]
Average correlation across all outer folds: 0.19093623705651203
Early stopping, best iteration is:
[1]	train's l1: 0.771604	valid's l1: 0.774018


C:\Users\simen\AppData\Local\Temp\ipykernel_41008\1244612948.py:72: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = pearsonr(y_test_outer, y_pred_outer)


Results: [0.09296440785335845, nan, nan, nan, nan, nan, nan, nan, nan, nan]
Average correlation across all outer folds: nan


[W 2024-09-26 12:16:33,010] Trial 14 failed with parameters: {'n_estimators': 455, 'learning_rate': 0.08749284596778963, 'max_depth': 8, 'num_leaves': 2000, 'reg_lambda': 35, 'reg_alpha': 70, 'bagging_fraction': 0.2, 'bagging_freq': 10, 'feature_fraction': 0.9} because of the following error: The value nan is not acceptable.
[W 2024-09-26 12:16:33,011] Trial 14 failed with value nan.
